## 1.라이브러리 선언 

In [1]:
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

## 2.이미지 전처리함수 선언 

In [2]:
sample = cv2.imread("E:/test_image.jpg",cv2.IMREAD_COLOR)
def imgRefining(inputImage):
    IMG_SIZE = 28

    ##1. 그레이컬러 변경
    sample_cvtColor = cv2.cvtColor(inputImage,cv2.COLOR_BGR2GRAY)
    sample_cvtSize = cv2.resize(sample_cvtColor,(IMG_SIZE,IMG_SIZE))
    sample_cvtNorm = sample_cvtSize.astype(float)/255.0
    sample_cvtShape = sample_cvtNorm.reshape(1,IMG_SIZE,IMG_SIZE,1)
    
    return sample_cvtShape

In [3]:
imgRefining(sample).shape

(1, 28, 28, 1)

## 3.모델 불러오기 

In [4]:
from tensorflow.keras.models import model_from_json
import warnings
warnings.filterwarnings(action="ignore")

## model 불러오기 (json, h5) 

json_file = open("E:/model_mnist_koppo.json", "r") 

loaded_model_json = json_file.read()

json_file.close()

loaded_model = model_from_json(loaded_model_json)

loaded_model.summary()

loaded_model.load_weights("E:/model_mnist_kopo_weight.h5")

C:\Users\SMART-24\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
W1102 14:40:03.765349  4536 nn_ops.py:4283] Large dropout rate: 0.8 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 32)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 32)          9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 3, 3, 32)          0         
_________________________________________________________________
flatten (Flatten)            (None, 288)               0

## 4.이미지캡쳐

In [5]:
cap = cv2.VideoCapture(0)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
CAP_SIZE = 150

In [ ]:
# ##말로 설명해줌 -:> python version update 필용 3버젼으로
# import pyttsx3
# engine = pyttsx3.init("dummy")

In [ ]:
## spacebar & 실시간
while(True):
    ret,colorFrame = cap.read()
    if ret == False:
        break
    
    ##캡쳐 영역설정
    ##좌표 수학적 계산
    x1= int(width/2-CAP_SIZE/2)
    y1= int(height/2-CAP_SIZE/2)
    x2= int(width/2+CAP_SIZE/2)
    y2= int(height/2+CAP_SIZE/2)
    
    cv2.rectangle(colorFrame,(x1,y1),(x2,y2),(0,0,255),3)
    ##예측용으로 만든것
    imputFrame = colorFrame.copy()
    
    ##사각형 이미지만 잡히게 할 수 있는 부분
    img_roi = colorFrame[y1:y1+CAP_SIZE, x1:x1+CAP_SIZE]
    
    refinedImage = imgRefining(img_roi)
    predictValue = np.argmax(loaded_model.predict(refinedImage))
    
    try:
        cv2.putText(colorFrame, str(predictValue),
                                    (x1,y1-10),
                                    cv2.FONT_HERSHEY_SIMPLEX,
                                    1,
                                    (0,255,255),
                                    3)
    except Exception as e:
        print(e)
        pass
    
    cv2.imshow("webcam",colorFrame)
    
    k = cv2.waitKey(33)
    
    if k == 27: #esc키
        cap.release()
        cv2.destroyAllWindows()
    elif k==32: #spacebar키
        ##예측
        refinedImage = imgRefining(img_roi)
        predictValue = np.argmax(loaded_model.predict(refinedImage))
#         engine.say(str(predictValue))
#         engine.runAndWait()
        print(predictValue)
        cv2.imwrite("E:/cap_test.jpg",img_roi)

cap.release()
cv2.destroyAllWindows()

In [ ]:
##감정 인지 할때 마지막에 이용해야함(이미지 캡쳐에서)
emotionList = ["angry","disgust","fear","happy","sad","surprise"]
emotionList[0]